<h1>Introduction</h1>
A research group in the Department of Computer Science at UBC has set up a Minecraft server to study how people play video games. They collect data in two main data frames: players.csv, which contains a list of all unique players along with demographic and player-level information, and sessions.csv, which records individual play sessions for each player, including session-level details.

In this project, we focus on the first dataset, players.csv, and investigate the question: “What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between different types of players?"
Finally, we decided to predict whether the players' subscribe the Video Game based on their age and played time.

<h3>Addressing Question 1</h3>

**What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

<h3>The Specific Question</h3>

Can a player’s total playtime and age predict whether they subscribe to the newsletter?

<h3>Variables need to used</h3>

Response variable (y): subscribe

Explanatory variables (x): Age, played_hours 

<h3> How the Data Helps Address the Specific Question</h3>

This dataset provides the three information needed for this question, which are **total playtime**, **age**, and **subscription status** for each player. By focusing on these variables and removing missing values in Age, I can examine how playtime and age differ between subscribers and non-subscribers.

<h1> Importing Data </h1>

In [ ]:
library(tidyverse)
url1='https://drive.google.com/uc?export=download&id=1LGq5BTBDuA8hTXNmLyc1A0I6Q0ifIL6e'
players<-read_csv(url1)
head(players)

In [ ]:
players_mean<- players|>
    summarise(mean_played_hours = round(mean(played_hours, na.rm = TRUE), 2),
              min_played_hours = min(played_hours, na.rm = TRUE),
              max_played_hours = max(played_hours, na.rm = TRUE),
              missing_played_hours = round(mean(is.na(played_hours)) * 100, 2),
              mean_age = round(mean(Age, na.rm = TRUE), 2),
              min_age = min(Age, na.rm = TRUE),
              max_age = max(Age, na.rm = TRUE),
              missing_age = round(mean(is.na(Age)) * 100, 2))
players_mean

<h2> Data Description: </h2>

<h3>Summary</h3>

This project analyzes data from the players.csv dataset, which contains information about 196 individual players on a Minecraft server, including their experience level, newsletter subscriptifon status, hashed email address, playtime, name, gender, and age.

The dataset includes seven variables. The table below summarizes each variable and its type.

<h3>Variables</h3>

 Variable Name | Type | Description | Example Value |
|----------------|------|--------------|----------------|
| `experience` | Categorical (`chr`) | Player’s skill level or rank. | `Pro` |
| `subscribe` | Boolean (`lgl`) | Indicates whether the player has an active subscription (TRUE) or not (FALSE). | `TRUE` |
| `hashedEmail` | String (`chr`) | Unique anonymized identifier for each player. | `f6daba4...` |
| `played_hours` | Numeric (`dbl`) | Total number of hours the player has spent playing. | `30.3` |
| `name` | String (`chr`) | Player’s first name. | `Morgan` |
| `gender` | Categorical (`chr`) | Player’s gender identity | `male` |
| `Age` | Numeric (`dbl`) | Player’s age in years. Contains some missing values (`NA`). | `17` |

---

<h3>Summary Statistic</h3>

| Variable | Mean | Min | Max | Missing (%) |
|-----------|------|-----------|------|--------------|
| `played_hours` | *5.85* | *0* | *223.1* | 0% |
| `Age` | *21.14* | *9* | *58* | 1.02% |
---

<h3>Direct Observations and Problems</h3>

+ **Experience:** character variable. It shows the players experience level. The categories — “Amateur,” “Regular,” “Veteran,” and “Pro” — indicate increasing levels of game proficiency in that order, and it could be useful in predicting playtime

+ **subscribe:** logical variable. It indicates whether a player subscribed to the game.'True' means the player did subscribe the game, 'False' means 
the player did not subscribe the game.

+ **hashedEmail:** character variable. It is a anonnymized version of the player's email, but is not relevant for analysis.

+ **played_hours :** numeric variable. It shows the number of hours the player has spent on the game.

+ **name:** character variable. It is the display name chosen by each player in the game.

+ **gender:** character variable. It contains has many different responses, such as “Other”, “Two-Spirited”, “Prefer not to say”, etc, showing the gender of each player.

+ **Age：** numeric variable. It shows the player's age. It has some missing values, which must be handled before modelling

<h3>Other Potential Issue</h3>

- The data may not represent all types of players (for example, older players or casual players may be missing).
  
- Data by self-recording (such as age) may contain errors.

- The Age variable contains NA values. This may cause errors in many functions.

- The hashedEmail variable is not human-readable, it may be useless.

- If we want to predict subscribe, and it is not stored as a factor, we need to convert it first before running any classifier.

<h3>How the Data be Collected</h3>

<p> A research group in Computer Science at UBC, led by Frank Wood, is collecting data about how people play video games. They have set up a Minecraft serverLinks to an external site., and players' actions are recorded as they navigate through the world. </p >

## wrangling step：
+ select the subscribe, Age, and played_hours variables
+ convert subscribe from a character variable into a factor

In [ ]:
players_clean<- players |> mutate(subscribe = as.factor(subscribe)) |> 
select(subscribe, Age, played_hours) |>
drop_na(subscribe, Age, played_hours)
head(players_clean)

## Visualization

In [ ]:
visualization_age_subscribe<- players_clean|>
            ggplot(aes(x = Age, fill = subscribe)) +
            geom_histogram(position = "identity", alpha = 0.3, binwidth = 5) +
            labs(title = "Age vs Subscribe",
                 x = "Age (years)",
                 y = "Count (per person)")
visualization_age_subscribe

visualization_playtime_subscribe<- players_clean|>
            ggplot(aes(x = played_hours, fill = subscribe)) +
            geom_histogram(position = "identity", alpha = 0.3, binwidth = 5) +
            labs(title = "Played_hours vs Subscribe",
                 x = "played_hours (h)",
                 y = "Count (per person)")
visualization_playtime_subscribe

We chose to create these two histograms to examine whether subscribers exhibit distinct patterns in age and total playtime. These two graphs show how subscribed and non-subscribed players are distributed across age and total playtime. By comparing the overlap between the two groups, we can see a clear pattern in the age distribution: subscribers tend to be younger, with most of them falling between 13 and 17 years old. A similar pattern appears in the playtime distribution. Subscribers are more concentrated in the lower playtime range, especially between 0 and 5 total hours. This suggests that players who subscribe to the newsletter are generally newer or lighter users of the server.

In [ ]:
Age_played_hours <-players_clean |> 
  ggplot(aes(x = Age, y = played_hours, color = subscribe)) +
  geom_point(alpha = 0.6) +
  labs(title = "Age vs Played Hours Colored by Subscription Status",
    x = "Age (years)",
    y = "Played Hours") 
Age_played_hours

From the played_hours side, players who don’t subscribe usually have 0 hours played. But relying only on played_hours is not accurate, because some subscribed users barely play at all, and some non-subscribed users actually play for a long time. This led me to consider whether age might also influence a player’s decision to subscribe.

From the age side, both subscribed and non-subscribed players appear across almost all age groups, but most subscribed players are under 30 years old.
